In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import numpy as np
from PIL import Image

def load_images_from_dlp_competition(noisy_dir, gt_dir):
    noisy_images = []
    clean_images = []
    image_ids = []

    # Sort to ensure alignment
    noisy_filenames = sorted(os.listdir(noisy_dir))
    gt_filenames = sorted(os.listdir(gt_dir))

    for noisy_name, gt_name in zip(noisy_filenames, gt_filenames):
        noisy_path = os.path.join(noisy_dir, noisy_name)
        gt_path = os.path.join(gt_dir, gt_name)

        # Load both images
        noisy_img = Image.open(noisy_path).convert('RGB')
        gt_img = Image.open(gt_path).convert('RGB')

        # Convert to NumPy arrays
        noisy_array = np.array(noisy_img)
        gt_array = np.array(gt_img)

        noisy_images.append(noisy_array)
        clean_images.append(gt_array)
        image_ids.append(noisy_name)

    return np.array(noisy_images), np.array(clean_images), image_ids


In [ ]:
###EDA
train_noisy_folder = '/kaggle/input/dlp-jan-2025-nppe-3/archive/train/train'
train_gt_folder = '/kaggle/input/dlp-jan-2025-nppe-3/archive/train/gt'

val_noisy_folder = '/kaggle/input/dlp-jan-2025-nppe-3/archive/val/val'   # path to noisy low-res images
val_gt_folder = '/kaggle/input/dlp-jan-2025-nppe-3/archive/val/gt'         # path to clean high-res ground truth images

noisy_imgs, clean_imgs, ids = load_images_from_dlp_competition(train_noisy_folder, train_gt_folder)
val_noisy_imgs, val_clean_imgs, ids = load_images_from_dlp_competition(val_noisy_folder, val_gt_folder)

print(f"Loaded train {len(noisy_imgs)} image pairs.")
print(f"Val Noisy image shape: {noisy_imgs[0].shape}, Ground truth shape: {clean_imgs[0].shape}")

print(f"Loaded Val {len(val_noisy_imgs)} image pairs.")
print(f"Val Noisy image shape: {val_noisy_imgs[0].shape}, Ground truth shape: {val_clean_imgs[0].shape}")


In [ ]:
import matplotlib.pyplot as plt

def plot_sample_pairs(noisy_imgs, clean_imgs, image_ids, n=3):
    plt.figure(figsize=(12, 4 * n))
    for i in range(n):
        # Noisy
        plt.subplot(n, 2, 2*i + 1)
        plt.imshow(noisy_imgs[i])
        plt.title(f"Noisy Image: {image_ids[i]}")
        plt.axis('off')
        
        # Ground Truth
        plt.subplot(n, 2, 2*i + 2)
        plt.imshow(clean_imgs[i])
        plt.title("Ground Truth")
        plt.axis('off')
        
    plt.tight_layout()
    plt.show()

plot_sample_pairs(noisy_imgs, clean_imgs, ids)


In [ ]:
import seaborn as sns

def plot_intensity_comparison(noisy_img, clean_img):
    noisy_gray = np.array(Image.fromarray(noisy_img).convert('L')).flatten()
    clean_gray = np.array(Image.fromarray(clean_img).convert('L')).flatten()

    plt.figure(figsize=(10, 4))
    sns.histplot(noisy_gray, label='Noisy', color='orange', kde=True)
    sns.histplot(clean_gray, label='Ground Truth', color='blue', kde=True)
    plt.title("Pixel Intensity Distribution (Grayscale)")
    plt.xlabel("Pixel Value")
    plt.legend()
    plt.show()

plot_intensity_comparison(noisy_imgs[0], clean_imgs[0])


In [ ]:
def summarize_images(imgs, label):
    imgs_flat = imgs.astype(np.float32).reshape(len(imgs), -1)
    print(f"--- {label} Image Stats ---")
    print("Mean:", np.mean(imgs_flat))
    print("Std Dev:", np.std(imgs_flat))
    print("Min:", np.min(imgs_flat))
    print("Max:", np.max(imgs_flat))
    print()

summarize_images(noisy_imgs, "Noisy")
summarize_images(clean_imgs, "Ground Truth")


In [ ]:
def visualize_noise_map(noisy, clean):
    noisy_resized = np.array(Image.fromarray(noisy).resize(clean.shape[1::-1], Image.BICUBIC))
    diff = np.abs(noisy_resized.astype(int) - clean.astype(int))

    plt.figure(figsize=(12, 4))
    plt.subplot(1, 3, 1)
    plt.imshow(noisy_resized)
    plt.title("Upscaled Noisy")

    plt.subplot(1, 3, 2)
    plt.imshow(clean)
    plt.title("Ground Truth")

    plt.subplot(1, 3, 3)
    plt.imshow(diff)
    plt.title("Difference Map")
    plt.show()

visualize_noise_map(noisy_imgs[0], clean_imgs[0])


In [ ]:
import math
import cv2

def calculate_psnr(img1, img2):
    mse = np.mean((img1.astype(np.float32) - img2.astype(np.float32)) ** 2)
    if mse == 0:
        return float('inf')
    PIXEL_MAX = 255.0
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

# Example:
noisy_upscaled = np.array(Image.fromarray(noisy_imgs[0]).resize(clean_imgs[0].shape[1::-1], Image.BICUBIC))
psnr_value = calculate_psnr(noisy_upscaled, clean_imgs[0])
print(f"PSNR between upscaled noisy and ground truth: {psnr_value:.2f} dB")


In [ ]:
train_noisy= noisy_imgs
train_gt =clean_imgs

val_noisy= val_noisy_imgs
val_gt= val_clean_imgs

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor, Compose

transform = Compose([ToTensor()])

class DLPDataset(Dataset):
    def __init__(self, noisy_imgs, clean_imgs):
        self.noisy_imgs = noisy_imgs
        self.clean_imgs = clean_imgs

    def __len__(self):
        return len(self.noisy_imgs)

    def __getitem__(self, idx):
        noisy = transform(Image.fromarray(self.noisy_imgs[idx])) * 255.0
        clean = transform(Image.fromarray(self.clean_imgs[idx])) * 255.0
        return noisy, clean

train_dataset = DLPDataset(train_noisy, train_gt)
val_dataset = DLPDataset(val_noisy, val_gt)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)


In [ ]:
import torch.nn.functional as F
import math

def calculate_batch_psnr(output, target):
    mse = F.mse_loss(output, target, reduction='none')
    mse = mse.view(mse.size(0), -1).mean(dim=1)
    psnr = 20 * torch.log10(1.0 / torch.sqrt(mse))
    return psnr.mean().item()


In [ ]:
!git clone --depth=1 https://github.com/sanghyun-son/EDSR-PyTorch.git

In [ ]:
import os

# Check if EDSR-PyTorch was cloned
print("EDSR-PyTorch" in os.listdir())

In [ ]:
# Change directory into the repo
os.chdir("EDSR-PyTorch")

# List contents to verify
print(os.listdir())


In [ ]:
import sys
sys.path.append('./src')  # Not 'code', it's 'src'

In [ ]:
import os
print(os.listdir('./src'))

In [ ]:
import sys
import os

# Step 1: Prevent argparse from picking up unwanted notebook args
sys.argv = ['']  # 🛑 Clear CLI args before importing

# Step 2: Add src directory to path
sys.path.append('./src')

# Step 3: Import EDSR and args safely
from model.edsr import EDSR
from option import args  # No more SystemExit!

# Step 4: Customize args
args.scale = [4]
args.n_resblocks = 8
args.n_feats = 64
args.rgb_range = 255
args.res_scale = 1
args.n_colors = 3

# Step 5: Create the model
model = EDSR(args).cuda()

In [ ]:
import torch
import torch.nn as nn
from tqdm import tqdm
from torch.nn.utils import clip_grad_norm_

# Setup
model = EDSR(args).cuda()
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scaler = torch.cuda.amp.GradScaler()
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)  # Halve LR every 5 epochs

best_psnr = 0
save_path = "best_model.pth"

def calculate_psnr(pred, target, max_val=255.0):
    mse = nn.functional.mse_loss(pred, target)
    psnr = 20 * torch.log10(max_val / torch.sqrt(mse + 1e-8))
    return psnr.item()

# Training Loop
num_epochs = 20
model.train()

for epoch in range(num_epochs):
    total_loss = 0
    loop = tqdm(train_loader, desc=f"Training Epoch {epoch+1}")

    for noisy, clean in loop:
        noisy = noisy.cuda()
        clean = clean.cuda()

        optimizer.zero_grad()
        with torch.amp.autocast("cuda"):  # AMP-enabled
            output = model(noisy)

            # Debug: check output range
            if torch.any(torch.isnan(output)) or torch.any(torch.isinf(output)):
                print("⚠️ NaN or Inf in model output!")

            # Optional: clamp output values
            # output = torch.clamp(output, 0, 255)

            loss = criterion(output, clean)

        scaler.scale(loss).backward()
        clip_grad_norm_(model.parameters(), max_norm=1.0)
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()
        loop.set_postfix(loss=loss.item())

    scheduler.step()
    print(f"[Epoch {epoch+1}] Training Loss: {total_loss / len(train_loader):.4f}")

    # Validation
    model.eval()
    val_psnr = 0
    with torch.no_grad():
        for noisy, clean in tqdm(val_loader, desc="Validating"):
            noisy = noisy.cuda()
            clean = clean.cuda()
            output = model(noisy)
            val_psnr += calculate_psnr(output, clean)

    val_psnr /= len(val_loader)
    print(f"[Epoch {epoch+1}] Validation PSNR: {val_psnr:.2f} dB")

    if val_psnr > best_psnr:
        best_psnr = val_psnr
        torch.save(model.state_dict(), save_path)
        print(f"✅ Saved new best model with PSNR: {best_psnr:.2f} dB")

    model.train()

# Load best model
model.load_state_dict(torch.load(save_path))
print("🔁 Loaded best model for inference.")


In [ ]:
pwd

In [ ]:
from PIL import Image

# Path to the image
image_path = '/kaggle/input/dlp-jan-2025-nppe-3/archive/test/test_00001.png'

# Open and check the image size
with Image.open(image_path) as img:
    print("Image size:", img.size)  # (width, height)


In [ ]:
import os
from PIL import Image
import numpy as np

def load_test_images(test_folder):
    filenames = sorted(os.listdir(test_folder))
    images = []

    for fname in filenames:
        img_path = os.path.join(test_folder, fname)
        img = Image.open(img_path).convert('RGB')
        images.append(np.array(img))

    return images, filenames

test_folder = "/kaggle/input/dlp-jan-2025-nppe-3/archive/test"
test_images, test_filenames = load_test_images(test_folder)


In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor

class TestDataset(Dataset):
    def __init__(self, images):
        self.images = images
        self.transform = ToTensor()

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.transform(Image.fromarray(self.images[idx])) * 255.0
        return img

test_dataset = TestDataset(test_images)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)


In [ ]:
pwd

In [ ]:
from torchvision.transforms import ToPILImage
import torch

model.eval()
os.makedirs("outputs", exist_ok=True)
to_pil = ToPILImage()

with torch.no_grad():
    idx = 0
    for batch in test_loader:
        batch = batch.cuda()
        preds = model(batch).clamp(0, 255)

        for i in range(preds.size(0)):
            img = to_pil(preds[i].cpu() / 255.0)  # Scale [0,255] → [0,1] for PIL
            out_name = test_filenames[idx].replace(".jpg", ".png")  # match your original
            img.save(f"outputs/{out_name}")
            idx += 1


In [ ]:
def images_to_csv(folder_path, output_csv):
    data_rows = []
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            image_path = os.path.join(folder_path, filename)
            image = Image.open(image_path).convert('L') 
            image_array = np.array(image).flatten()[::8]
            # Replace 'test_' with 'gt_' in the ID
            image_id = filename.split('.')[0].replace('test_', 'gt_')
            data_rows.append([image_id, *image_array])
    column_names = ['ID'] + [f'pixel_{i}' for i in range(len(data_rows[0]) - 1)]
    df = pd.DataFrame(data_rows, columns=column_names)
    df.to_csv(output_csv, index=False)
    print(f'Successfully saved to {output_csv}')

In [ ]:
folder_path = 'outputs'
output_csv = '/kaggle/working/submission.csv'
images_to_csv(folder_path, output_csv)

In [ ]:
import pandas as pd

# Load submission.csv
submission_df = pd.read_csv('/kaggle/working/submission.csv')

# Show basic info
print(submission_df.info())

# Show first few rows
submission_df.head()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load submission CSV
df = submission_df

# Basic Info
print("Shape:", df.shape)
print("Columns:", df.columns[:10], "...")  # Show some column names
print("Sample IDs:", df['ID'].head().tolist())

# Check pixel value range
pixel_columns = df.columns[1:]
all_pixels = df[pixel_columns].values.flatten()
print("Pixel stats → Min:", np.min(all_pixels), "Max:", np.max(all_pixels), "Mean:", np.mean(all_pixels))

# Plot a few sample images
def plot_sample_images(df, num_images=4):
    fig, axs = plt.subplots(1, num_images, figsize=(15, 5))
    for i in range(num_images):
        img_data = df.iloc[i, 1:].values.astype(np.uint8)
        # Reshape to known size: (640, 1024) → (2560, 4096) before downsampling ::8 → back to 640x1024 // 8 = 81920
        img = img_data.reshape(256, 320)  # 256*320 = 81920
        axs[i].imshow(img, cmap='gray')
        axs[i].set_title(df.iloc[i, 0])
        axs[i].axis('off')
    plt.tight_layout()
    plt.show()


plot_sample_images(df)

# Histogram of pixel intensities
plt.figure(figsize=(8, 4))
plt.hist(all_pixels, bins=50, color='steelblue', edgecolor='black')
plt.title("Pixel Intensity Distribution")
plt.xlabel("Pixel Value")
plt.ylabel("Frequency")
plt.grid(True)
plt.show()


In [ ]:
###using esrgan

In [ ]:
from PIL import Image

# Path to the image
image_path = '/kaggle/working/EDSR-PyTorch/outputs/test_00001.png'

# Open and check the image size
with Image.open(image_path) as img:
    print("Image size:", img.size)  # (width, height)

In [ ]:
%cd /kaggle/working


In [ ]:
!rm -rf Real-ESRGAN


In [ ]:
!git clone https://github.com/xinntao/Real-ESRGAN
%cd Real-ESRGAN


In [ ]:
!pip install basicsr facexlib gfpgan
!pip install -r requirements.txt
!python setup.py develop



In [ ]:
pwd

In [ ]:
ls

In [ ]:
!ls scripts


In [ ]:
!mkdir -p experiments/pretrained_models


In [ ]:
!wget https://huggingface.co/lllyasviel/Annotators/resolve/main/RealESRGAN_x4plus.pth -P experiments/pretrained_models


In [ ]:
!pip install --upgrade torchvision


In [ ]:
!sed -i 's/from torchvision.transforms.functional_tensor import rgb_to_grayscale/from torchvision.transforms.functional import rgb_to_grayscale/' /usr/local/lib/python3.10/dist-packages/basicsr/data/degradations.py


In [ ]:
!python inference_realesrgan.py \
-n RealESRGAN_x4plus \
-i /kaggle/working/EDSR-PyTorch/outputs\
--face_enhance




In [ ]:
!find . -type f -name "*.png"



In [ ]:
from PIL import Image

# Path to the image
image_path = './results/test_00020_out.png'

# Open and check the image size
with Image.open(image_path) as img:
    print("Image size:", img.size)  # (width, height)


In [ ]:
'''
def images_to_csv(folder_path, output_csv):
    data_rows = []
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            image_path = os.path.join(folder_path, filename)
            image = Image.open(image_path).convert('L') 
            image_array = np.array(image).flatten()[::8]
            # Replace 'test_' with 'gt_' in the ID
            image_id = filename.split('.')[0].replace('test_', 'gt_')
            data_rows.append([image_id, *image_array])
    column_names = ['ID'] + [f'pixel_{i}' for i in range(len(data_rows[0]) - 1)]
    df = pd.DataFrame(data_rows, columns=column_names)
    df.to_csv(output_csv, index=False)
    print(f'Successfully saved to {output_csv}')
'''

In [ ]:

import os
import numpy as np
import pandas as pd
from PIL import Image

def images_to_csv1(folder_path, output_csv):
    data_rows = []
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            image_path = os.path.join(folder_path, filename)
            image = Image.open(image_path).convert('L').resize((1024, 640))
            image_array = np.array(image).flatten()[::8]
            
            # Extract ID from something like 'test_00043_out.png'
            base = os.path.splitext(filename)[0]  # test_00043_out
            parts = base.split('_')               # ['test', '00043', 'out']
            if len(parts) >= 2:
                image_id = f"gt_{parts[1]}"       # gt_00043
            else:
                image_id = base                   # fallback if unexpected format
            
            data_rows.append([image_id, *image_array])
    
    column_names = ['ID'] + [f'pixel_{i}' for i in range(len(data_rows[0]) - 1)]
    df = pd.DataFrame(data_rows, columns=column_names)
    df.to_csv(output_csv, index=False)
    print(f'Successfully saved to {output_csv}')

# Call the function
folder_path = './results/'
output_csv = '/kaggle/working/submission_1.csv'
images_to_csv1(folder_path, output_csv)


In [ ]:
import pandas as pd

# Load submission.csv
submission_df = pd.read_csv('/kaggle/working/submission_1.csv')

# Show basic info
print(submission_df.info())

# Show first few rows
submission_df.head()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load submission CSV
df = submission_df

# Basic Info
print("Shape:", df.shape)
print("Columns:", df.columns[:10], "...")  # Show some column names
print("Sample IDs:", df['ID'].head().tolist())

# Check pixel value range
pixel_columns = df.columns[1:]
all_pixels = df[pixel_columns].values.flatten()
print("Pixel stats → Min:", np.min(all_pixels), "Max:", np.max(all_pixels), "Mean:", np.mean(all_pixels))

# Plot a few sample images
def plot_sample_images(df, num_images=4):
    fig, axs = plt.subplots(1, num_images, figsize=(15, 5))
    for i in range(num_images):
        img_data = df.iloc[i, 1:].values.astype(np.uint8)
        # Reshape to known size: (640, 1024) → (2560, 4096) before downsampling ::8 → back to 640x1024 // 8 = 81920
        img = img_data.reshape(256, 320)  # 256*320 = 81920
        axs[i].imshow(img, cmap='gray')
        axs[i].set_title(df.iloc[i, 0])
        axs[i].axis('off')
    plt.tight_layout()
    plt.show()


plot_sample_images(df)

# Histogram of pixel intensities
plt.figure(figsize=(8, 4))
plt.hist(all_pixels, bins=50, color='steelblue', edgecolor='black')
plt.title("Pixel Intensity Distribution")
plt.xlabel("Pixel Value")
plt.ylabel("Frequency")
plt.grid(True)
plt.show()


In [ ]:
###Using swinIR

In [ ]:
!git clone https://github.com/JingyunLiang/SwinIR.git
%cd SwinIR

In [ ]:
!mkdir -p experiments/pretrained_models

!wget https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x4_GAN.pth -P experiments/pretrained_models
!wget https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/004_grayDN_DFWB_s128w8_SwinIR-M_noise15.pth -P experiments/pretrained_models
!wget https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/004_grayDN_DFWB_s128w8_SwinIR-M_noise25.pth -P experiments/pretrained_models
!wget https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/004_grayDN_DFWB_s128w8_SwinIR-M_noise50.pth -P experiments/pretrained_models
!wget https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/005_colorDN_DFWB_s128w8_SwinIR-M_noise15.pth -P experiments/pretrained_models
!wget https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/005_colorDN_DFWB_s128w8_SwinIR-M_noise25.pth -P experiments/pretrained_models
!wget https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/005_colorDN_DFWB_s128w8_SwinIR-M_noise50.pth -P experiments/pretrained_models
!wget https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/006_CAR_DFWB_s126w7_SwinIR-M_jpeg10.pth -P experiments/pretrained_models
!wget https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/006_CAR_DFWB_s126w7_SwinIR-M_jpeg20.pth -P experiments/pretrained_models
!wget https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/006_CAR_DFWB_s126w7_SwinIR-M_jpeg30.pth -P experiments/pretrained_models
!wget https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/006_CAR_DFWB_s126w7_SwinIR-M_jpeg40.pth -P experiments/pretrained_models


In [ ]:
!ls experiments/pretrained_models

In [ ]:
from torchvision.transforms import Resize, Compose, Grayscale, ToTensor
from torch.utils.data import Dataset, DataLoader

transform = Compose([
    Grayscale(num_output_channels=1),
    Resize((128, 128)),
    ToTensor()
])

class DLPDataset1(Dataset):
    def __init__(self, noisy_imgs, clean_imgs):
        self.noisy_imgs = noisy_imgs
        self.clean_imgs = clean_imgs

    def __len__(self):
        return len(self.noisy_imgs)

    def __getitem__(self, idx):
        noisy = transform(Image.fromarray(self.noisy_imgs[idx])) * 255.0
        clean = transform(Image.fromarray(self.clean_imgs[idx])) * 255.0
        return noisy, clean

train_dataset = DLPDataset1(train_noisy, train_gt)
val_dataset = DLPDataset1(val_noisy, val_gt)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)


In [ ]:
!pip install basicsr

In [ ]:
!sed -i 's/from torchvision.transforms.functional_tensor import rgb_to_grayscale/from torchvision.transforms.functional import rgb_to_grayscale/' /usr/local/lib/python3.10/dist-packages/basicsr/data/degradations.py


In [ ]:
from basicsr.archs.swinir_arch import SwinIR
import torch
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = SwinIR(
    upscale=1,
    in_chans=1,
    img_size=128,  # adjust if your patch size is different
    window_size=8,
    img_range=255.0,
    depths=[6, 6, 6, 6, 6, 6],
    embed_dim=180,
    num_heads=[6, 6, 6, 6, 6, 6],
    mlp_ratio=2,
    upsampler='',  # for denoising
    resi_connection='1conv'
).to(device)

criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
num_epochs = 10

In [ ]:
from tqdm import tqdm
from skimage.metrics import peak_signal_noise_ratio as compare_psnr
from skimage.metrics import structural_similarity as compare_ssim

def train_one_epoch(model, dataloader, criterion, optimizer):
    model.train()
    total_loss = 0.0
    for noisy, clean in tqdm(dataloader, desc="Training"):
        noisy, clean = noisy.to(device), clean.to(device)
        optimizer.zero_grad()
        output = model(noisy)
        loss = criterion(output, clean)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)

def validate(model, dataloader, criterion):
    model.eval()
    total_loss = 0.0
    total_psnr = 0.0
    total_ssim = 0.0
    torch.cuda.empty_cache()  # ← clear unused memory
    
    with torch.no_grad():
        for noisy, clean in tqdm(dataloader, desc="Validating"):
            noisy, clean = noisy.to(device), clean.to(device)
            output = model(noisy)

            loss = criterion(output, clean)
            total_loss += loss.item()

            output_np = output.cpu().numpy()
            clean_np = clean.cpu().numpy()

            # Loop over batch
            for i in range(output_np.shape[0]):
                out_img = np.squeeze(output_np[i])
                gt_img = np.squeeze(clean_np[i])

                psnr = compare_psnr(gt_img, out_img, data_range=255.0)
                ssim = compare_ssim(gt_img, out_img, data_range=255.0)

                total_psnr += psnr
                total_ssim += ssim

    n = len(dataloader.dataset)
    return (
        total_loss / len(dataloader),
        total_psnr / n,
        total_ssim / n
    )



In [ ]:
best_psnr = 0.0

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    
    train_loss = train_one_epoch(model, train_loader, criterion, optimizer)
    val_loss, val_psnr, val_ssim = validate(model, val_loader, criterion)

    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | PSNR: {val_psnr:.2f} | SSIM: {val_ssim:.4f}")
    
    if val_psnr > best_psnr:
        best_psnr = val_psnr
        torch.save(model.state_dict(), f"best_swinir_psnr_{best_psnr:.2f}.pth")
        print(f"✅ Saved new best model at Epoch {epoch+1} with PSNR: {val_psnr:.2f}")


In [ ]:
###inference
from torchvision.transforms import Resize, Compose, Grayscale, ToTensor
from torch.utils.data import Dataset, DataLoader
from PIL import Image

test_transform = Compose([
    Grayscale(num_output_channels=1),
    Resize((1024, 640)),
    ToTensor()
])

class TestDataset(Dataset):
    def __init__(self, noisy_imgs):
        self.noisy_imgs = noisy_imgs

    def __len__(self):
        return len(self.noisy_imgs)

    def __getitem__(self, idx):
        noisy = test_transform(Image.fromarray(self.noisy_imgs[idx])) * 255.0
        return noisy

test_images, test_filenames = load_test_images(test_folder)

test_dataset = TestDataset(test_images)  # not test_noisy
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)


In [ ]:
from torchvision.transforms import ToPILImage
import torch

# --- Load best model ---
model.load_state_dict(torch.load("/kaggle/working/SwinIR/best_swinir_psnr_43.99.pth"))
model.eval()

# --- Inference & Save ---
os.makedirs("outputs1", exist_ok=True)
to_pil = ToPILImage()

with torch.no_grad():
    for idx, batch in enumerate(test_loader):
        batch = batch.cuda()
        preds = model(batch).clamp(0, 255)

        # Convert tensor to image and save
        img = to_pil(preds[0].cpu() / 255.0)  # Normalize for PIL
        out_name = test_filenames[idx].rsplit(".", 1)[0] + ".png"
        img.save(f"outputs1/{out_name}")

In [ ]:
from PIL import Image

# Path to the image
image_path = '/kaggle/working/SwinIR/outputs1/test_00001.png'

# Open and check the image size
with Image.open(image_path) as img:
    print("Image size:", img.size)  # (width, height)

In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image

def images_to_csv3(folder_path, output_csv):
    data_rows = []
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            image_path = os.path.join(folder_path, filename)
            image = Image.open(image_path).convert('L').resize((1024, 640))
            image_array = np.array(image).flatten()[::8]  # downsample

            # Extract ID from something like 'test_00043_SwinIR.png'
            base = os.path.splitext(filename)[0]      # test_00043_SwinIR
            parts = base.split('_')                   # ['test', '00043', 'SwinIR']
            if len(parts) >= 2 and parts[1].isdigit():
                image_id = f"gt_{parts[1]}"           # gt_00043
            else:
                image_id = base

            data_rows.append([image_id, *image_array])
    
    if not data_rows:
        print("No valid images found.")
        return

    column_names = ['ID'] + [f'pixel_{i}' for i in range(len(data_rows[0]) - 1)]
    df = pd.DataFrame(data_rows, columns=column_names)
    df.to_csv(output_csv, index=False)
    print(f'Successfully saved to {output_csv}')

# Call the function
folder_path = '/kaggle/working/SwinIR/outputs1'
output_csv = '/kaggle/working/submission_3.csv'
images_to_csv3(folder_path, output_csv)


In [ ]:
import pandas as pd

# Load submission.csv
submission_df = pd.read_csv('/kaggle/working/submission_3.csv')

# Show basic info
print(submission_df.info())

# Show first few rows
submission_df.head()

In [ ]:
from IPython.display import FileLink
FileLink('/kaggle/working/submission_3.csv')
import zipfile

with zipfile.ZipFile("/kaggle/working/submission.zip", 'w') as zipf:
    zipf.write("/kaggle/working/submission_3.csv", arcname="submission_3.csv")


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load submission CSV
df = submission_df

# Basic Info
print("Shape:", df.shape)
print("Columns:", df.columns[:10], "...")  # Show some column names
print("Sample IDs:", df['ID'].head().tolist())

# Check pixel value range
pixel_columns = df.columns[1:]
all_pixels = df[pixel_columns].values.flatten()
print("Pixel stats → Min:", np.min(all_pixels), "Max:", np.max(all_pixels), "Mean:", np.mean(all_pixels))

# Plot a few sample images
def plot_sample_images(df, num_images=4):
    fig, axs = plt.subplots(1, num_images, figsize=(15, 5))
    for i in range(num_images):
        img_data = df.iloc[i, 1:].values.astype(np.uint8)
        # Reshape to known size: (640, 1024) → (2560, 4096) before downsampling ::8 → back to 640x1024 // 8 = 81920
        img = img_data.reshape(256, 320)  # 256*320 = 81920
        axs[i].imshow(img, cmap='gray')
        axs[i].set_title(df.iloc[i, 0])
        axs[i].axis('off')
    plt.tight_layout()
    plt.show()


plot_sample_images(df)

# Histogram of pixel intensities
plt.figure(figsize=(8, 4))
plt.hist(all_pixels, bins=50, color='steelblue', edgecolor='black')
plt.title("Pixel Intensity Distribution")
plt.xlabel("Pixel Value")
plt.ylabel("Frequency")
plt.grid(True)
plt.show()
